In [1]:
import numpy as np
import ase
from ase.build import molecule
from ase.optimize import LBFGS

In [2]:
from qmlearn.api.api4ase import QMLCalculator
from qmlearn.drivers.mol import QMMol
from qmlearn.io.model import db2qmmodel

In [3]:
qmmodel = db2qmmodel('h2o_vib_QML_set.hdf5')

Guess DB names : {'qmmol': 'rks/qmmol', 'atoms': 'rks/train_atoms_36', 'properties': 'rks/train_props_36'}
Guess mmodels: {'gamma': KernelRidge(alpha=0.1), 'd_gamma': LinearRegression(), 'd_energy': LinearRegression(), 'd_forces': LinearRegression()}


In [4]:
second_learn = {
        'energy' : 'd_energy',
        'forces' : 'd_forces',
        }
calc = QMLCalculator(qmmodel = qmmodel, second_learn = second_learn, method = 'gamma')

In [5]:
atoms_test=molecule('H2O')
atoms_test.positions[0,0] += 0.2

In [6]:
atoms = atoms_test.copy()
atoms.calc = calc

In [7]:
LBFGS(atoms, trajectory = 'opt.traj').run(fmax=0.01)
atoms.positions

       Step     Time          Energy         fmax
LBFGS:    0 22:49:58    -2070.590437        1.2679
LBFGS:    1 22:49:58    -2070.617218        0.2239
LBFGS:    2 22:49:58    -2070.617600        0.1172
LBFGS:    3 22:49:59    -2070.618134        0.0966
LBFGS:    4 22:49:59    -2070.618393        0.0388
LBFGS:    5 22:49:59    -2070.618418        0.0058


array([[ 1.92626984e-01,  8.07108743e-06,  9.72790197e-02],
       [ 3.69296506e-03,  7.65848295e-01, -4.66036258e-01],
       [ 3.68009279e-03, -7.65856343e-01, -4.66074638e-01]])

In [8]:
basis = 'cc-pvTZ'
xc = 'lda,vwn_rpa'
method = 'rks'
charge = 0

In [9]:
atoms = atoms_test.copy()
refqmmol = QMMol(atoms = atoms, method = method, basis=basis, xc = xc, charge=charge)
atoms.calc = QMLCalculator(qmmodel = refqmmol, method = 'engine')

In [10]:
LBFGS(atoms, trajectory = 'opt.traj').run(fmax=0.01)
atoms.positions

       Step     Time          Energy         fmax
LBFGS:    0 22:50:00    -2070.596525        1.3259
LBFGS:    1 22:50:01    -2070.616301        0.2563
LBFGS:    2 22:50:01    -2070.617393        0.1126
LBFGS:    3 22:50:02    -2070.618015        0.1030
LBFGS:    4 22:50:03    -2070.618396        0.0495
LBFGS:    5 22:50:03    -2070.618425        0.0106
LBFGS:    6 22:50:04    -2070.618427        0.0005


array([[ 1.92577545e-01, -1.87575598e-14,  9.72371798e-02],
       [ 3.71122733e-03,  7.65804799e-01, -4.66034590e-01],
       [ 3.71122733e-03, -7.65804799e-01, -4.66034590e-01]])

In [11]:
qmmodel = db2qmmodel('./h2o_md_300_QML_set.hdf5')
atoms = atoms_test.copy()
atoms.calc = QMLCalculator(qmmodel = qmmodel, second_learn = second_learn, method = 'gamma')
LBFGS(atoms, trajectory = 'opt.traj').run(fmax=0.01)
atoms.positions

Guess DB names : {'qmmol': 'rks/qmmol', 'atoms': 'rks/train_atoms_30', 'properties': 'rks/train_props_30'}
Guess mmodels: {'gamma': KernelRidge(alpha=0.1), 'd_gamma': LinearRegression(), 'd_energy': LinearRegression(), 'd_forces': LinearRegression()}
       Step     Time          Energy         fmax
LBFGS:    0 22:50:06    -2070.596523        1.3260
LBFGS:    1 22:50:06    -2070.616324        0.2564
LBFGS:    2 22:50:06    -2070.617414        0.1126
LBFGS:    3 22:50:06    -2070.618032        0.1029
LBFGS:    4 22:50:06    -2070.618412        0.0495
LBFGS:    5 22:50:07    -2070.618440        0.0106
LBFGS:    6 22:50:07    -2070.618442        0.0014


array([[ 1.92610243e-01,  1.10151982e-05,  9.72291056e-02],
       [ 3.69627463e-03,  7.65794575e-01, -4.66026391e-01],
       [ 3.69348310e-03, -7.65805587e-01, -4.66034714e-01]])